In [1]:
import hail as hl

In [2]:
hl.init()

2022-11-21 21:14:51 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-11-21 21:14:52 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20221121-2114-0.2.99-57537fea08d4.log


In [3]:
maps_table = hl.read_table('gs://janucik-dataproc-stage/01_maps/1_array_rerun_maps_per_variant_main_21_Nov_22_v1/02a_f_maps_table.ht')

In [4]:
maps_table.show()

+----------------------+
| lof_csq_collapsed    |
+----------------------+
| str                  |
+----------------------+
| "HC"                 |
| "LC"                 |
| "missense_variant"   |
| "synonymous_variant" |
+----------------------+

+------------------------------------------------------------------------------+
| N_singletons                                                                 |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [98660,97002,54519,4376,931,4879,138,1630,33824,8965,8962,14417,66715,647... |
| [11631,11349,6300,509,195,789,32,186,4216,1307,1373,2058,8356,7962,4718,2... |
| [2090809,2059323,1205397,93382,41045,172978,5325,38230,802213,268638,2747... |
| [931260,916526,548740,43764,26872,98257,3091,19008,382794,140973,147438,2... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| expected_singletons                                                          |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [7.62e+04,7.62e+04,3.81e+04,3.25e+03,6.73e+02,3.18e+03,2.99e+02,1.51e+03,... |
| [1.05e+04,1.05e+04,5.11e+03,4.81e+02,2.09e+02,6.74e+02,1.02e+02,3.01e+02,... |
| [1.93e+06,1.93e+06,9.61e+05,9.14e+04,4.10e+04,1.38e+05,2.01e+04,5.82e+04,... |
| [9.31e+05,9.31e+05,4.63e+05,5.10e+04,3.12e+04,8.93e+04,1.64e+04,3.78e+04,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| N_variants                                                                   |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [152811,152811,78756,6936,1421,6711,632,3224,52715,11935,12058,21615,9788... |
| [20382,20382,10263,994,426,1402,200,624,7668,2010,2197,3664,13827,13129,7... |
| [3913394,3913394,2034940,201045,88371,302746,42458,127251,1550621,424443,... |
| [1942562,1942562,1017656,116902,70713,205315,36283,86090,842660,247820,27... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| ps_agg                                                                       |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [6.46e-01,6.35e-01,6.92e-01,6.31e-01,6.55e-01,7.27e-01,2.18e-01,5.06e-01,... |
| [5.71e-01,5.57e-01,6.14e-01,5.12e-01,4.58e-01,5.63e-01,1.60e-01,2.98e-01,... |
| [5.34e-01,5.26e-01,5.92e-01,4.64e-01,4.64e-01,5.71e-01,1.25e-01,3.00e-01,... |
| [4.79e-01,4.72e-01,5.39e-01,3.74e-01,3.80e-01,4.79e-01,8.52e-02,2.21e-01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| maps                                                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [1.47e-01,1.36e-01,2.08e-01,1.62e-01,1.82e-01,2.54e-01,-2.54e-01,3.69e-02... |
| [5.66e-02,4.28e-0

In [5]:
maps_table.freq_metas[0]

<ArrayExpression of type array<dict<str, str>>>

In [6]:
maps_zipped = maps_table.transmute(
data = hl.zip(maps_table.N_singletons, maps_table.expected_singletons, maps_table.N_variants, maps_table.ps_agg, maps_table.maps, maps_table.maps_sem, maps_table.freq_metas[0])
)

In [7]:
maps_zipped.show()

+----------------------+
| lof_csq_collapsed    |
+----------------------+
| str                  |
+----------------------+
| "HC"                 |
| "LC"                 |
| "missense_variant"   |
| "synonymous_variant" |
+----------------------+

+------------------------------------------------------------------------------+
| data                                                                         |
+------------------------------------------------------------------------------+
| array<tuple(int64, float64, int64, float64, float64, float64, dict<str, s... |
+------------------------------------------------------------------------------+
| [(98660,7.62e+04,152811,6.46e-01,1.47e-01,1.22e-03,{"group":"adj"}),(9700... |
| [(11631,1.05e+04,20382,5.71e-01,5.66e-02,3.47e-03,{"group":"adj"}),(11349... |
| [(2090809,1.93e+06,3913394,5.34e-01,4.17e-02,2.52e-04,{"group":"adj"}),(2... |
| [(931260,9.31e+05,1942562,4.79e-01,1.20e-16,3.58e-04,{"group":"adj"}),(91... |
+------------------------------------------------------------------------------+

In [68]:
maps_expl = maps_zipped.explode('data')

In [71]:
maps_expl.show()

,
lof_csq_collapsed,data
str,"tuple(int64, float64, int64, float64, float64, float64, dict<str, str>)"
"""HC""","(98660,7.62e+04,152811,6.46e-01,1.47e-01,1.22e-03,{""group"":""adj""})"
"""HC""","(97002,7.62e+04,152811,6.35e-01,1.36e-01,1.23e-03,{""group"":""raw""})"
"""HC""","(54519,3.81e+04,78756,6.92e-01,2.08e-01,1.64e-03,{""group"":""adj"",""pop"":""nfe""})"
"""HC""","(4376,3.25e+03,6936,6.31e-01,1.62e-01,5.79e-03,{""group"":""adj"",""pop"":""fin""})"
"""HC""","(931,6.73e+02,1421,6.55e-01,1.82e-01,1.26e-02,{""group"":""adj"",""pop"":""mid""})"
"""HC""","(4879,3.18e+03,6711,7.27e-01,2.54e-01,5.44e-03,{""group"":""adj"",""pop"":""oth""})"
"""HC""","(138,2.99e+02,632,2.18e-01,-2.54e-01,1.64e-02,{""group"":""adj"",""pop"":""ami""})"
"""HC""","(1630,1.51e+03,3224,5.06e-01,3.69e-02,8.81e-03,{""group"":""adj"",""pop"":""asj""})"


In [93]:
maps_clean = maps_expl.annotate(
Singletons = maps_expl.data[0],
Expected_Singletons = maps_expl.data[1],
N_variants = maps_expl.data[2],
PS = maps_expl.data[3],
MAPS = maps_expl.data[4],
MAPS_Sem = maps_expl.data[5],
Metadata = maps_expl.data[6].values()
)

In [94]:
maps_clean.show()

,,,,,,,,
lof_csq_collapsed,data,Singletons,Expected_Singletons,N_variants,PS,MAPS,MAPS_Sem,Metadata
str,"tuple(int64, float64, int64, float64, float64, float64, dict<str, str>)",int64,float64,int64,float64,float64,float64,array<str>
"""HC""","(98660,7.62e+04,152811,6.46e-01,1.47e-01,1.22e-03,{""group"":""adj""})",98660,7.62e+04,152811,6.46e-01,1.47e-01,1.22e-03,"[""adj""]"
"""HC""","(97002,7.62e+04,152811,6.35e-01,1.36e-01,1.23e-03,{""group"":""raw""})",97002,7.62e+04,152811,6.35e-01,1.36e-01,1.23e-03,"[""raw""]"
"""HC""","(54519,3.81e+04,78756,6.92e-01,2.08e-01,1.64e-03,{""group"":""adj"",""pop"":""nfe""})",54519,3.81e+04,78756,6.92e-01,2.08e-01,1.64e-03,"[""adj"",""nfe""]"
"""HC""","(4376,3.25e+03,6936,6.31e-01,1.62e-01,5.79e-03,{""group"":""adj"",""pop"":""fin""})",4376,3.25e+03,6936,6.31e-01,1.62e-01,5.79e-03,"[""adj"",""fin""]"
"""HC""","(931,6.73e+02,1421,6.55e-01,1.82e-01,1.26e-02,{""group"":""adj"",""pop"":""mid""})",931,6.73e+02,1421,6.55e-01,1.82e-01,1.26e-02,"[""adj"",""mid""]"
"""HC""","(4879,3.18e+03,6711,7.27e-01,2.54e-01,5.44e-03,{""group"":""adj"",""pop"":""oth""})",4879,3.18e+03,6711,7.27e-01,2.54e-01,5.44e-03,"[""adj"",""oth""]"
"""HC""","(138,2.99e+02,632,2.18e-01,-2.54e-01,1.64e-02,{""group"":""adj"",""pop"":""ami""})",138,2.99e+02,632,2.18e-01,-2.54e-01,1.64e-02,"[""adj"",""ami""]"
"""HC""","(1630,1.51e+03,3224,5.06e-01,3.69e-02,8.81e-03,{""group"":""adj"",""pop"":""asj""})",1630,1.51e+03,3224,5.06e-01,3.69e-02,8.81e-03,"[""adj"",""asj""]"


In [95]:
maps_clean.count()

2392